Importation des librairies


In [124]:
import numpy as np

Class de definition de repère

In [113]:
# Calculer l'inverse d'une matrice 

def invert_matrix(mat):
    """
    Retourne l'inverse de `mat` ou lève une exception si non inversible.
    """
    try:
        return np.linalg.inv(mat)
    except np.linalg.LinAlgError:
        raise ValueError("Matrice singulière : impossible d'appliquer l'inverse.")

class RepereLocal:
    def __init__(self, origine, rotation_matrix=None, nom=None):
        self.origine = np.array(origine)  # Position de du repère dans W
        # Matrice de rotation 3x3 (identité si non spécifiée)
        self.rotation = rotation_matrix if rotation_matrix is not None else np.eye(3)
        self.nom = "nom" if nom is not None else "Repère Local"
    
    def get_transform_matrixs(self):
        """Retourne la matrice de transformation homogène 4x4"""
        T = np.eye(4)
        T[0:3, 0:3] = self.rotation
        T[0:3, 3] = self.origine
        T_inv = invert_matrix(T)
        return [ T , T_inv]

    



In [ ]:
def transform_point_W_to_C(point_W, repere_C):
        point = point_W.reshape(4,1)
        [ T , T_inv]= repere_C.get_transform_matrixs()
        point_in_C = T_inv @ point
    
        return point_in_C


In [122]:
# 1. Définir le repère monde W (repère de référence global)
Repere_W = RepereLocal(origine=[0, 0, 0], nom="Repère W")


In [123]:
Repere_C = RepereLocal(origine=[-16, 16, 20], rotation_matrix=np.array([[1/np.sqrt(2), 0.5, 0.5],
                                                                    [1/np.sqrt(2), -0.5, -0.5],
                                                                    [0., 1/np.sqrt(2), -1/np.sqrt(2)]]),
                                                                    nom="Repère C")
Repere_C.get_transform_matrixs()

[array([[  0.70710678,   0.5       ,   0.5       , -16.        ],
        [  0.70710678,  -0.5       ,  -0.5       ,  16.        ],
        [  0.        ,   0.70710678,  -0.70710678,  20.        ],
        [  0.        ,   0.        ,   0.        ,   1.        ]]),
 array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
        [ 0.5       , -0.5       ,  0.70710678,  1.85786438],
        [ 0.5       , -0.5       , -0.70710678, 30.14213562],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])]

In [117]:
point_w = np.array([-2.25,11,1.5,1]).reshape(4,1)
#point.reshape(4,1)

In [118]:
point_C = transform_point_W_to_C(point_w, Repere_C)

point_C

array([[ 6.18718434],
       [-3.70647545],
       [22.45647545],
       [ 1.        ]])

array([[ 6.18718434],
       [-3.70647545],
       [22.45647545],
       [ 1.        ]])

In [120]:
# 6. FONCTION : Transformer M du repère W vers le repère C


In [121]:
# 4. Créer un repère C dans le monde W
# C est décalé de (2, 3, 1) et tourné de 45° autour de l'axe Z
angle = np.radians(45)
rotation_C = np.array([
    [np.cos(angle), -np.sin(angle), 0],
    [np.sin(angle), np.cos(angle), 0],
    [0, 0, 1]
])
C = RepereLocal(origine=[2, 3, 1], rotation_matrix=rotation_C)

# 5. Définir un point M dans le repère monde W
M_in_W = np.array([5, 7, 2])



# 7. Calculer les coordonnées de M dans C
M_in_C = transform_point_W_to_C(M_in_W, C)

# Afficher les résultats
print(f"Repère monde W: origine à {W.origine}")
print(f"Repère C: origine à {C.origine} dans W")
print(f"Angle de rotation C: 45° autour de Z")
print(f"\nPoint M dans le repère W: {M_in_W}")
print(f"Point M dans le repère C: {M_in_C}")
print(f"\nCoordonnées arrondies: [{M_in_C[0]:.2f}, {M_in_C[1]:.2f}, {M_in_C[2]:.2f}]")

ValueError: cannot reshape array of size 3 into shape (4,1)

In [ ]:
#M_in_W
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [ ]:

class Camera:
    """
    Classe représentant une caméra avec sa matrice intrinsèque K.
    Permet de projeter des points 3D vers le repère caméra et l'image.
    """
    
    def __init__(self, K, R_camera=None):
        """
        Initialise la caméra.
        
        Args:
            K (np.ndarray): Matrice intrinsèque 3x3
            R (np.ndarray): Matrice de rotation 3x3 (optionnel)
            t (np.ndarray): Vecteur de translation 3x1 (optionnel)
        """
        self.K = K  # Matrice intrinsèque
        self.R_camera = R_camera if R_camera is not None else np.eye(4)  # Rotation (identité par défaut)
        #self.t = t if t is not None else np.zeros((3, 1))  # Translation (zéro par défaut)
    
    def monde_vers_camera(self, point_3d):
        """
        Transforme un point 3D du monde vers le repère caméra.
        
        Args:
            point_3d (np.ndarray): Point 3D (3,) ou (3,1)
        
        Returns:
            np.ndarray: Point dans le repère caméra
        """
        point_3d = point_3d.reshape(4, 1) if point_3d.ndim == 1 else point_3d
        point_camera = self.R @ point_3d + self.t
        return point_camera.flatten()
    
    def camera_vers_image(self, point_camera):
        """
        Projette un point du repère caméra vers le plan image.
        
        Args:
            point_camera (np.ndarray): Point 3D dans le repère caméra
        
        Returns:
            np.ndarray: Coordonnées 2D dans l'image (x, y)
        """
        point_camera = point_camera.reshape(3, 1) if point_camera.ndim == 1 else point_camera
        
        # Projection perspective
        point_image_h = self.K @ point_camera
        
        # Conversion en coordonnées inhomogènes
        point_image = point_image_h[:2] / point_image_h[2]
        return point_image.flatten()
    
    def monde_vers_image(self, point_3d):
        """
        Projette directement un point 3D du monde vers l'image.
        
        Args:
            point_3d (np.ndarray): Point 3D du monde
        
        Returns:
            np.ndarray: Coordonnées 2D dans l'image
        """
        point_camera = self.monde_vers_camera(point_3d)
        return self.camera_vers_image(point_camera)







Point dans le repère caméra: [1.  0.5 5. ]
Coordonnées dans l'image: [480. 320.]


In [ ]:
angle = np.radians(45)
rotation_C = np.array([
    [np.cos(angle), -np.sin(angle), 0],
    [np.sin(angle), np.cos(angle), 0],
    [0, 0, 1]
])

In [ ]:

# Exemple d'utilisation
if __name__ == "__main__":
    # Définir la matrice intrinsèque
    K = np.array([
        [800, 0, 320],
        [0, 800, 240],
        [0, 0, 1]
    ])
    
    # Créer une caméra
    camera = Camera(K)
    
    # Point 3D du monde
    point_3d = np.array([1, 0.5, 5])
    
    # Obtenir les coordonnées dans le repère caméra
    point_camera = camera.monde_vers_camera(point_3d)
    print(f"Point dans le repère caméra: {point_camera}")
    
    # Projeter sur l'image
    point_image = camera.monde_vers_image(point_3d)
    print(f"Coordonnées dans l'image: {point_image}")

In [ ]:
# 2. Définis


Repère monde W: origine à [0 0 0]
Repère C: origine à [2 3 1] dans W
Angle de rotation C: 45° autour de Z

Point M dans le repère W: [5 7 2]
Point M dans le repère C: [4.94974747 0.70710678 1.        ]

Coordonnées arrondies: [4.95, 0.71, 1.00]
